# Imports

`numpy`      package for array handling  
`pandas`     import to create and modify dataframes     
`matplotlib` package for data visulization  
`seaborn`    build on matploblib, higher level graph functions

In [1]:
# Imports 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

# Loading the data
Here we are taking the inbuilt function of keras to load the data from the server  
The dataset file in present in the [Link to dataset in amazon server](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz)  
The inbuilt code 
```python
def load_data():
    """Loads CIFAR10 dataset.
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
    dirname = 'cifar-10-batches-py'
    origin = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
    path = get_file(dirname, origin=origin, untar=True)

    num_train_samples = 50000

    x_train = np.empty((num_train_samples, 3, 32, 32), dtype='uint8')
    y_train = np.empty((num_train_samples,), dtype='uint8')

    for i in range(1, 6):
        fpath = os.path.join(path, 'data_batch_' + str(i))
        (x_train[(i - 1) * 10000: i * 10000, :, :, :],
         y_train[(i - 1) * 10000: i * 10000]) = load_batch(fpath)

    fpath = os.path.join(path, 'test_batch')
    x_test, y_test = load_batch(fpath)

    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))

    if K.image_data_format() == 'channels_last':
        x_train = x_train.transpose(0, 2, 3, 1)
        x_test = x_test.transpose(0, 2, 3, 1)

    return (x_train, y_train), (x_test, y_test)
```
## Regarding Dataset
cifar10(Canadian Institute For Advanced Research) is dataset of 10 objectst namely

| Label | Description   |
|:----  |:------------- | 
|0      | airplane      | 
|1      | automobile    | 
|2      | bird          |
|3      | cat           |
|4      | deer          |
|5      | dog           |
|6      | frog          |
|7      | horse         |
|8      | ship          |
|9      | truck         |

[official Link](https://www.cs.toronto.edu/~kriz/cifar.html)   
`Line 4` : created a dictionary with respective labels

In [4]:
# Loading the dataset
from keras.datasets import boston_housing
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()
columns = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV']

Exception: URL fetch failure on https://s3.amazonaws.com/keras-datasets/boston_housing.npz: None -- [Errno 11002] getaddrinfo failed

# Understand the dataset
Let see the size and shape of test and training tuples

``On Execution``  
There will be 50000 samples of 28*28 resolution of images in training set  
There will be 10000 samples of 28*28 resolution of images in testing set

In [5]:
# Understanding the data
print("The number of training samples",len(x_train))
print("The number of testing samples",len(x_test))
print("The shape of training samples array",np.shape(x_train))
print("The shape of training samples labels", np.shape(y_train))

NameError: name 'x_train' is not defined

# Visualizing the data
Let us try to visualise the few samples of data so, we could get an idea of how the data looks like  
`On Execution`  
We can see ten images of 

In [7]:
# Visualizing the data

df = pd.DataFrame(x_train, columns=columns[:-1])
df.head()

## Labels 
print("{} values :".format(columns[-1]),y_train[0:10]) 

NameError: name 'x_train' is not defined

# Preprocessing the data

## Vairables
`image_width`,`image_height`,`image_channels` would describe the dimentions of the image  
`classes` to detemine how many catogories of samples present in out dataset. By nature mnist have 0-9 images to ten classes  

## Creating sparse vector representation
`to_categorical` is converting into one hot encoding. Means each vector is represented by one hot encoding.
0 --> [1,0,0,0,0,0,0,0,0,0]   
1 --> [0,1,0,0,0,0,0,0,0,0]  
2 --> [0,0,1,0,0,0,0,0,0,0]   
and similarly goes on 

`np.expand_dims` would just increst one dimentions in the end. Like .... [1,2,3] to [[1],[2],[3]]  

## Normalization
`Line16`,`Line17`is normalization we are divinding all the pixal values by 255. so all the numerical values are converted between 0 and 1
>Note: since its a simple dataset there is not much of processing required to attain good accuracies. For all real time datasest preprocessing like normalizing , standadising , on hot encoding, filling the missing values, transforming features, feeding the data in batches and all other type of preprocessing is required


In [8]:
# Preprocessing the data

## Variables
image_width = 32
image_height = 32
image_channels = 3 
image_shape = (image_width,image_height,image_channels)

## Normalization
mean = x_train.mean(axis=0)        #Train Data
std = x_train.std(axis=0)
train_data = (x_train - mean) / std

mean = y_train.mean(axis=0)        #Train Labels Data
std = y_train.std(axis=0)
y_train = (y_train - mean) / std

mean = x_test.mean(axis=0)        #Test Data
std = x_test.std(axis=0)
train_data = (x_test - mean) / std

mean = y_test.mean(axis=0)        #Test Labels Data
std = y_test.std(axis=0)
y_test = (y_test - mean) / std


NameError: name 'x_train' is not defined

# Training varibles
These Training varbles are hyper parameters for neural network training.   
`epochs` : each epoch is forward propagation + backward propagation over the whole dataset once is called one epoch.  
`learning_rate` : the magnitude in which the weights are modified one the acquired loss.   
`learning_rate_decay` : there can be high leanring rate at the beining of the training when the loss is high. Over a period of time the learning rate can reduce for fine training of network.  
`batch_size` : the data is fed to the network in batches of 32 samples at each time. This batch feeding is done all over the whole dataset.  

In [9]:
# Training varibles
learning_rate = 0.0001
learning_rate_decay = 0.000001
batch_size = 32
epochs = 20
classes = 10

# Neural Netowork Model
`Line 6` : we are building a keras sequential model  
`Line 32` : we are using stochastic gradient decent optimizer  
`Line 36` : compiling the model to check if the model is build properly.  

The loss function being used is `categorical_crossentropy` since its a multi class classification


In [10]:
# Building the model
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop

model = Sequential()
model.add(Dense(64, activation = 'relu',input_shape=(x_train.shape[1],)))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1))

# Optimizer
optimizer = RMSprop(lr=learning_rate, rho=0.9, epsilon=None, decay=learning_rate_decay)

# Compiling the model
model.compile(loss='mse',optimizer=optimizer,metrics=['mae'])


NameError: name 'x_train' is not defined

# Training 
Training is the process of feeding the data to neural network and modifiying the weights of the model using the the backpropagation algorithm. The backpropagation using loss the function acquires the loss over batch size of data and does a backpropagation to modify the weights in such a way the in the next epoch the loss would be less when compared to the current epoch

In [11]:
# Training
model_history = model.fit(x_train, y_train, epochs=epochs,validation_split=0.2, verbose=1)

NameError: name 'x_train' is not defined

# Results
using the trained model we try to predict what are the values of images in the test set

In [12]:
# Results
y_pred = model.predict(x=x_test, batch_size=batch_size, verbose=1)

NameError: name 'x_test' is not defined

# Verifying the results
cheking the results how good they are with the first 10 samples.   
Plotting the graphs of test and train set accuracies and loss values. 
> NOTE: This plot is a very curicial step. These plots would tell us how good the model converges and if there is any overfitting

In [13]:
# Verifying the results
print("Ground truths of first 10 images in test set",np.array(y_test[0:10]))
print("Predicted values of first 10 image in test set",np.argmax(y_pred[0:10],axis=1))

loss = model_history.history['loss']
val_loss = model_history.history['val_loss']
plt.plot(loss,label='train')
plt.plot(val_loss,label='test')
plt.title('loss Graph')
plt.ylabel('value')
plt.xlabel('epochs')
plt.legend()
plt.show()

acc = model_history.history['mean_absolute_error']
val_acc = model_history.history['val_mean_absolute_error']
plt.plot(acc,label='train')
plt.plot(val_acc,label='test')
plt.title('mean_absolute_error Graph')
plt.ylabel('value')
plt.xlabel('epochs')
plt.legend()
plt.show()

NameError: name 'y_test' is not defined